In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('/home/tebrigabby/python/data_files/train.csv', index_col='Id') 
X_test = pd.read_csv('/home/tebrigabby/python/data_files/test (1).csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [88]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [89]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [90]:
# Step 1: Drop columns with categorical data
# Fill in the lines below: drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [91]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


In [92]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


In [93]:
'''Notice that the 'Condition2' column in the validation data contains the values 'RRAn' and 'RRNn', but these don't appear 
in the training data -- thus, if we try to use an ordinal encoder with scikit-learn, the code will throw an error.
Run the code cell below to save the problematic columns to a Python list bad_label_cols. Likewise, columns that can be safely ordinal 
encoded are stored in good_label_cols.'''

"Notice that the 'Condition2' column in the validation data contains the values 'RRAn' and 'RRNn', but these don't appear \nin the training data -- thus, if we try to use an ordinal encoder with scikit-learn, the code will throw an error.\nRun the code cell below to save the problematic columns to a Python list bad_label_cols. Likewise, columns that can be safely ordinal \nencoded are stored in good_label_cols."

In [94]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Condition2', 'Functional', 'RoofMatl']


In [95]:
# You should ordinal encode the categorical columns in good_label_cols.
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

In [109]:
label_X_train[good_label_cols]

KeyError: "['Functional'] not in index"

In [96]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


In [97]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [98]:
# Fill in the line below: How many categorical variables in the training data
# have cardinality greater than 10?
high_cardinality_numcols = 3

# Fill in the line below: How many columns are needed to one-hot encode the 
# 'Neighborhood' variable in the training data?
num_cols_neighborhood = 25


In [99]:
# create a one-hot encoding for columns with cardinality less than 10.
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'Exterior2nd', 'Exterior1st']


In [100]:
# Step 4: One-hot encoding
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)


/home/tebrigabby/.local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [101]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084


In [124]:
# Drop the specified columns from X_test
columns_to_drop = ['Condition2', 'Functional', 'RoofMatl']

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in label_X_test.columns if col not in columns_to_drop]


In [125]:
# Drop categorical columns that will not be encoded
ordinal_encoder = OrdinalEncoder()
label_X_test[good_label_cols] = ordinal_encoder.fit_transform(label_X_test[good_label_cols])

In [126]:
label_X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,3.0,11694,1.0,3.0,3.0,0.0,4.0,0.0,16.0,...,108,0,0,260,0,0,7,2007,6.0,5.0
871,20,3.0,6600,1.0,3.0,3.0,0.0,4.0,0.0,12.0,...,0,0,0,0,0,0,8,2009,8.0,4.0
93,30,3.0,13360,1.0,0.0,1.0,0.0,4.0,0.0,6.0,...,0,44,0,0,0,0,8,2009,8.0,4.0
818,20,3.0,13265,1.0,0.0,3.0,0.0,1.0,0.0,11.0,...,59,0,0,0,0,0,7,2008,8.0,4.0
303,20,3.0,13704,1.0,0.0,3.0,0.0,0.0,0.0,5.0,...,81,0,0,0,0,0,1,2006,8.0,4.0


In [127]:
label_X_test.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Functional
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.0,2.0,777.0,1.0,3.0,3.0,0.0,4.0,0.0,12.0,...,0.0,0.0,18.0,0.0,0.0,5.0,4.0,8.0,4.0,6.0
1462,0.0,3.0,964.0,1.0,0.0,3.0,0.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,24.0,5.0,4.0,8.0,4.0,6.0
1463,5.0,3.0,947.0,1.0,0.0,3.0,0.0,4.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,2.0,4.0,8.0,4.0,6.0
1464,5.0,3.0,587.0,1.0,0.0,3.0,0.0,4.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,5.0,4.0,8.0,4.0,6.0
1465,11.0,3.0,118.0,1.0,0.0,1.0,0.0,4.0,0.0,22.0,...,0.0,0.0,27.0,0.0,0.0,0.0,4.0,8.0,4.0,6.0


In [131]:
label_X_test_dropped = label_X_test.drop(columns=['Functional'])

In [133]:
label_X_test_dropped_filled = label_X_test_dropped.fillna(label_X_test_dropped.mean())

In [135]:
label_X_test_dropped_filled.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.0,2.0,777.0,1.0,3.0,3.0,0.0,4.0,0.0,12.0,...,0.0,0.0,0.0,18.0,0.0,0.0,5.0,4.0,8.0,4.0
1462,0.0,3.0,964.0,1.0,0.0,3.0,0.0,0.0,0.0,12.0,...,24.0,0.0,0.0,0.0,0.0,24.0,5.0,4.0,8.0,4.0
1463,5.0,3.0,947.0,1.0,0.0,3.0,0.0,4.0,0.0,8.0,...,22.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,8.0,4.0
1464,5.0,3.0,587.0,1.0,0.0,3.0,0.0,4.0,0.0,8.0,...,24.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,8.0,4.0
1465,11.0,3.0,118.0,1.0,0.0,1.0,0.0,4.0,0.0,22.0,...,68.0,0.0,0.0,27.0,0.0,0.0,0.0,4.0,8.0,4.0


In [134]:
model=RandomForestRegressor(n_estimators=100,  random_state=0)
model.fit(label_X_train, y_train)

preds_test= model.predict(label_X_test_dropped_filled)
#output---------------

output=pd.DataFrame({'Id': label_X_test_dropped_filled.index,
               'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)